In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
import scipy
import scipy.io as sio
import h5py
import elm
import numpy as np
from os_elm import OS_ELM 

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def loadmat(filename, **kwargs):
    try: 
        return sio.loadmat(filename, **kwargs)
    except NotImplementedError:
        import h5py as h5
        mat = None
        with h5.File(filename, 'r') as f:
            mat = f['YYM_H'].value
        return mat

In [3]:
def data2seq(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return np.asarray(X)

def seq2data(seq, n_frames):
    X = list()
    for s in seq:
        data = []
        for ss in s:
            if data == []:
                data = ss
            else:
                data = data + ss
        data = data / n_frames
        X.append(data)
    return np.asarray(X)

def transform(data, n_frames):
    results = []
    l = -1
    result = []
    for d in data:
        d = np.expand_dims(d, 1).transpose(1, 0)
        if l == -1:
            l = d[0][0]
            result = d
        elif l == d[0][0]:
            result = np.concatenate((result, d))
        else:
            results.append(result)
            l = d[0][0]
            result = d
    results = np.asarray(results)
    
    datas = None
    for r in results: 
        seq = data2seq(r, n_frames)
        d = seq2data(seq, n_frames)
        if datas is None:
            datas = d
        else:
            datas = np.concatenate((datas, d))
    return datas

def generate_data(start, end):
    datas = None
    for i in range(start, end):
        fileName = './data/matfeatures/' + '{:03d}'.format(i) + 'feature_1.mat'
        print(fileName)
        data = loadmat(fileName)
        data = data.transpose(1, 0)
        data = transform(data, 16)
        if datas is None:
            datas = data
        else:
            datas = np.concatenate((datas, data), axis = 0)
    x = datas[:, 1:]
    y = datas[:, :1]
    y = y.reshape(y.shape[0]).astype(int)
    y_onehot = one_hot(y, 101)
    return x, y_onehot

def one_hot(y, classes):
    I = np.eye(classes)
    ret = []
    l = 0
    for y_ in y:
        ret.append(I[y_])
            
    return np.array(ret)

def load_train_data(x_train, y_train_onehot):
    border = 1000
    x_train_init = x_train[:border]
    y_train_init = y_train_onehot[:border]
    x_train_seq = x_train[border:]
    y_train_seq = y_train_onehot[border:]
    
    return x_train_init, y_train_init, x_train_seq, y_train_seq

In [4]:
os_elm = OS_ELM(
        n_input_nodes=1000,
        n_hidden_nodes=500,
        n_output_nodes=101,
        loss='mean_squared_error',
        activation='sigmoid',
        C = 2 ** 2,
    )

In [5]:
x_train_init, y_train_init = generate_data(1, 2)
x_test, y_test_onehot = generate_data(50, 64)

./data/matfeatures/001feature_1.mat


/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


./data/matfeatures/050feature_1.mat
./data/matfeatures/051feature_1.mat
./data/matfeatures/052feature_1.mat
./data/matfeatures/053feature_1.mat
./data/matfeatures/054feature_1.mat
./data/matfeatures/055feature_1.mat
./data/matfeatures/056feature_1.mat
./data/matfeatures/057feature_1.mat
./data/matfeatures/058feature_1.mat
./data/matfeatures/059feature_1.mat
./data/matfeatures/060feature_1.mat
./data/matfeatures/061feature_1.mat
./data/matfeatures/062feature_1.mat
./data/matfeatures/063feature_1.mat


In [7]:
y_train_init.shape

(16903, 101)

In [6]:
os_elm.init_train(x_train_init, y_train_init)

In [10]:
batch_size = 2048
for epoch in range(100):
    for index in range(2, 62):
        x_train_seq, y_train_seq = generate_data(index, index+1)
        for i in range(0, len(x_train_seq), batch_size):
            x_batch = x_train_seq[i:i+batch_size]
            y_batch = y_train_seq[i:i+batch_size]
            os_elm.seq_train(x_batch, y_batch)

        [loss, accuracy] = os_elm.evaluate(x_test, y_test_onehot, metrics=['loss', 'accuracy'])
        print('epoch: %d, index: %d, val_loss: %f, val_accuracy: %f' % (epoch, index, loss, accuracy))

./data/matfeatures/002feature_1.mat


/usr/lib/python3/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


epoch: 0, index: 2, val_loss: 31.396252, val_accuracy: 0.008964
./data/matfeatures/003feature_1.mat
epoch: 0, index: 3, val_loss: 24.129877, val_accuracy: 0.009404
./data/matfeatures/004feature_1.mat
epoch: 0, index: 4, val_loss: 21.090750, val_accuracy: 0.008959
./data/matfeatures/005feature_1.mat
epoch: 0, index: 5, val_loss: 19.622347, val_accuracy: 0.008848
./data/matfeatures/006feature_1.mat
epoch: 0, index: 6, val_loss: 13.661818, val_accuracy: 0.008459
./data/matfeatures/007feature_1.mat
epoch: 0, index: 7, val_loss: 9.772054, val_accuracy: 0.007527
./data/matfeatures/008feature_1.mat
epoch: 0, index: 8, val_loss: 8.690383, val_accuracy: 0.008519
./data/matfeatures/009feature_1.mat
epoch: 0, index: 9, val_loss: 4.479802, val_accuracy: 0.008699
./data/matfeatures/010feature_1.mat
epoch: 0, index: 10, val_loss: 3.554589, val_accuracy: 0.008823
./data/matfeatures/011feature_1.mat
epoch: 0, index: 11, val_loss: 3.156040, val_accuracy: 0.008930
./data/matfeatures/012feature_1.mat
epo

epoch: 1, index: 25, val_loss: 0.012341, val_accuracy: 0.015473
./data/matfeatures/026feature_1.mat
epoch: 1, index: 26, val_loss: 0.012172, val_accuracy: 0.016790
./data/matfeatures/027feature_1.mat
epoch: 1, index: 27, val_loss: 0.012111, val_accuracy: 0.015956
./data/matfeatures/028feature_1.mat
epoch: 1, index: 28, val_loss: 0.012039, val_accuracy: 0.013950
./data/matfeatures/029feature_1.mat
epoch: 1, index: 29, val_loss: 0.011976, val_accuracy: 0.014716
./data/matfeatures/030feature_1.mat
epoch: 1, index: 30, val_loss: 0.011902, val_accuracy: 0.015798
./data/matfeatures/031feature_1.mat
epoch: 1, index: 31, val_loss: 0.011738, val_accuracy: 0.014374
./data/matfeatures/032feature_1.mat
epoch: 1, index: 32, val_loss: 0.011615, val_accuracy: 0.014369
./data/matfeatures/033feature_1.mat
epoch: 1, index: 33, val_loss: 0.011563, val_accuracy: 0.015019
./data/matfeatures/034feature_1.mat
epoch: 1, index: 34, val_loss: 0.011537, val_accuracy: 0.013792
./data/matfeatures/035feature_1.mat


epoch: 2, index: 48, val_loss: 0.010519, val_accuracy: 0.017769
./data/matfeatures/049feature_1.mat
epoch: 2, index: 49, val_loss: 0.010512, val_accuracy: 0.017765
./data/matfeatures/050feature_1.mat
epoch: 2, index: 50, val_loss: 0.010514, val_accuracy: 0.017675
./data/matfeatures/051feature_1.mat
epoch: 2, index: 51, val_loss: 0.010518, val_accuracy: 0.018051
./data/matfeatures/052feature_1.mat
epoch: 2, index: 52, val_loss: 0.010509, val_accuracy: 0.018419
./data/matfeatures/053feature_1.mat
epoch: 2, index: 53, val_loss: 0.010510, val_accuracy: 0.017551
./data/matfeatures/054feature_1.mat
epoch: 2, index: 54, val_loss: 0.010513, val_accuracy: 0.018817
./data/matfeatures/055feature_1.mat
epoch: 2, index: 55, val_loss: 0.010503, val_accuracy: 0.020280
./data/matfeatures/056feature_1.mat
epoch: 2, index: 56, val_loss: 0.010504, val_accuracy: 0.020190
./data/matfeatures/057feature_1.mat
epoch: 2, index: 57, val_loss: 0.010494, val_accuracy: 0.019741
./data/matfeatures/058feature_1.mat


OSError: Can't read data (inflate() failed)